In [1]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 55.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=d064192ebc4f984330695d14e6d9202b0597d70b958a4a57cb1d8232c5a99fe3
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [17]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy2
from tqdm import tqdm
import re
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras import layers
from keras.models import Sequential,Model
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Machine Learning (ITHUB) /ДИ 2023/lessons/data/Tweets.csv')
df.dropna(inplace=True)
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [4]:
norm_text=[]
# 1 - только буквы
# 2 - перевод в нижний регистр
# 3 - привести все слова в единую форму
morph=pymorphy2.MorphAnalyzer()
for s in tqdm(df['text']):
    s1 = re.sub(r'[^\w\s]+|[\d]+', r'',s).strip()
    s1 = s1.lower()
    s1 = word_tokenize(s1)
    words=[]
    for i in s1:
        pv = morph.parse(i)
        words.append(pv[0].normal_form)
    sentence=' '.join(words)
    norm_text.append(sentence)

100%|██████████| 27480/27480 [00:14<00:00, 1893.87it/s]


In [5]:
df['norm_text']=norm_text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['norm_text'])

X = tokenizer.texts_to_sequences(df['norm_text'])

vocab_size = len(tokenizer.word_index) + 1

In [6]:
maxlen = 30
X = pad_sequences(X, padding='post', maxlen=maxlen)
print(X[0, :])

[285  16  68   1 119  47   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]


In [7]:
encoder = LabelEncoder() #кодируем каждую метку класса числом
y = encoder.fit_transform(df['sentiment'])
y = to_categorical(y)

In [8]:
X_train_full,X_test,y_train_full,y_test = train_test_split(X,y,test_size=0.2,random_state=0)
X_train,X_valid,y_train,y_valid = train_test_split(X_train_full,y_train_full,test_size=0.2,random_state=0)

In [30]:
model_lstm = Sequential()
model_lstm.add(layers.Embedding(input_dim=vocab_size,
                           output_dim=50,
                           input_length=maxlen))

model_lstm.add(layers.LSTM(64))
model_lstm.add(layers.Dense(y.shape[1], activation='softmax'))

model_lstm.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=["accuracy"])
model_lstm.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 30, 50)            1395700   
                                                                 
 lstm_1 (LSTM)               (None, 64)                29440     
                                                                 
 dense_13 (Dense)            (None, 3)                 195       
                                                                 
Total params: 1425335 (5.44 MB)
Trainable params: 1425335 (5.44 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
history=model_lstm.fit(X_train, y_train, epochs =10,batch_size=5,validation_data=(X_valid,y_valid))

acc=model_lstm.evaluate(X_test,y_test)
print(np.array(acc)[1].round(2))

Epoch 1/10
3518/3518 [==============================] - 136s 38ms/step - loss: 1.0686 - accuracy: 0.4167 - val_loss: 1.0735 - val_accuracy: 0.3564
Epoch 2/10
3518/3518 [==============================] - 131s 37ms/step - loss: 0.8240 - accuracy: 0.6267 - val_loss: 0.7459 - val_accuracy: 0.6850
Epoch 3/10
3518/3518 [==============================] - 130s 37ms/step - loss: 0.6303 - accuracy: 0.7468 - val_loss: 0.7017 - val_accuracy: 0.7043
Epoch 4/10
3518/3518 [==============================] - 132s 37ms/step - loss: 0.5388 - accuracy: 0.7951 - val_loss: 0.7351 - val_accuracy: 0.6952
Epoch 5/10
3518/3518 [==============================] - 130s 37ms/step - loss: 0.4669 - accuracy: 0.8257 - val_loss: 0.7856 - val_accuracy: 0.6980
Epoch 6/10
3518/3518 [==============================] - 131s 37ms/step - loss: 0.4040 - accuracy: 0.8533 - val_loss: 0.8401 - val_accuracy: 0.6882
Epoch 7/10
3518/3518 [==============================] - 131s 37ms/step - loss: 0.3453 - accuracy: 0.8746 - val_loss: 0

## Transformers

Теория - https://sysblok.ru/knowhow/kak-rabotajut-transformery-krutejshie-nejroseti-nashih-dnej/

Меньше теории - https://cloud.yandex.ru/blog/posts/2023/02/ml-transformers

Пример Keras - https://www.kaggle.com/code/quadeer15sh/transformers-for-text-classification

In [12]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, heads, neurons):
        super(TransformerEncoder, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=heads, key_dim=embed_dim)
        self.ffn = Sequential(
            [layers.Dense(neurons, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(0.5)
        self.dropout2 = layers.Dropout(0.5)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [32]:
heads = 2
neurons = 32

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, 50)
x = embedding_layer(inputs)
transformer_block = TransformerEncoder(50, heads, neurons)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.35)(x)
outputs = layers.Dense(y.shape[1], activation='softmax')(x)
model_tr = Model(inputs=inputs, outputs=outputs)

In [33]:
model_tr.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=["accuracy"])
model_tr.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 30)]              0         
                                                                 
 token_and_position_embeddi  (None, 30, 50)            1397200   
 ng_4 (TokenAndPositionEmbe                                      
 dding)                                                          
                                                                 
 transformer_encoder_4 (Tra  (None, 30, 50)            23832     
 nsformerEncoder)                                                
                                                                 
 global_average_pooling1d_4  (None, 50)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dropout_14 (Dropout)        (None, 50)                0   

In [34]:
model_name = "model.h5"
checkpoint = ModelCheckpoint(model_name,
                            monitor="val_loss",
                            mode="min",
                            save_best_only = True,
                            verbose=1)

earlystopping = EarlyStopping(monitor='val_loss', patience = 3, verbose = 1)


In [35]:
history = model_tr.fit(X_train,y_train,
                    validation_data=(X_valid,y_valid),
                    epochs=10,
                    batch_size=5,
                    callbacks=[earlystopping,checkpoint])

Epoch 1/10
3517/3518 [============================>.] - ETA: 0s - loss: 0.8216 - accuracy: 0.6280
Epoch 1: val_loss improved from inf to 0.76684, saving model to model.h5
3518/3518 [==============================] - 100s 27ms/step - loss: 0.8216 - accuracy: 0.6280 - val_loss: 0.7668 - val_accuracy: 0.6980
Epoch 2/10
3517/3518 [============================>.] - ETA: 0s - loss: 0.6392 - accuracy: 0.7406
Epoch 2: val_loss did not improve from 0.76684
3518/3518 [==============================] - 94s 27ms/step - loss: 0.6393 - accuracy: 0.7406 - val_loss: 0.7862 - val_accuracy: 0.7080
Epoch 3/10
3517/3518 [============================>.] - ETA: 0s - loss: 0.5621 - accuracy: 0.7801
Epoch 3: val_loss improved from 0.76684 to 0.76601, saving model to model.h5
3518/3518 [==============================] - 97s 28ms/step - loss: 0.5622 - accuracy: 0.7801 - val_loss: 0.7660 - val_accuracy: 0.6932
Epoch 4/10
3517/3518 [============================>.] - ETA: 0s - loss: 0.5097 - accuracy: 0.8064
Epoch

In [36]:
acc=model_tr.evaluate(X_test,y_test)
print(np.array(acc)[1].round(2))

172/172 [==============================] - 1s 8ms/step - loss: 1.0178 - accuracy: 0.6729
0.67
